<a href="https://colab.research.google.com/github/mananmehtagit/ipynb_notebooks/blob/main/ai_agent_topic_explorer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Purpose of this agent is scrape web for relevent articles and information based on input topic

We will create two tools the first tool is for searching the internet and getting URLs, and the second tool is for reading those URLs.


In [19]:
# !pip install -qU langchain-google-vertexai # use this for commercial use

!pip install langchain_google_genai duckduckgo_search #use google genai for prototype purpose
from google.colab import userdata

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.2 MB/s eta 0:00:00


In [24]:
# prompt: the first tool is for searching the internet and getting URLs, and the second tool is for reading those URLs.

from duckduckgo_search import DDGS
from langchain_google_genai import ChatGoogleGenerativeAI
import requests
from bs4 import BeautifulSoup

def search_ddgs_and_store(query, num_results=10):
  """Searches DuckDuckGo and stores results in an array.

  Args:
    query: The search query.
    num_results: The maximum number of results to retrieve.

  Returns:
    An array of search result dictionaries.
  """
  search_results = []
  results = DDGS().text(query, region='wt-wt', safesearch='Moderate', timelimit='y', max_results=num_results)  # fetch results from DDGS

  # Iterate through the results and extract relevant information
  for i,result in enumerate(results): # extract title, snippet, and URL
        search_results.append(result)

  return search_results



# summarize text from url
def summarize_url(url):
  """Reads a URL, fetches content, and creates a summary using Google AI LLM.

  Args:
    url: The URL to process.

  Returns:
    A summary of the URL content.
  """
  try:
    # Fetch the content of the URL
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad status codes

    # Parse the HTML content using Beautiful Soup
    soup = BeautifulSoup(response.content, "html.parser")

    # Extract the text content
    text = soup.get_text()

    # Initialize the Gemini model
    llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=userdata.get('GOOGLE_API_KEY'))

    # Generate the summary
    summary = llm.predict(f"Summarize the following text: {text}")

    return summary

  except requests.exceptions.RequestException as e:
    print(f"Error fetching URL {url}: {e}")
    return None
  except Exception as e:
    print(f"Error processing URL {url}: {e}")
    return None

# Example usage
topic = "What is the latest news on AI safety research?"
urls = search_ddgs_and_store(topic,3)  # Get the top 3 URLs
# print(urls)

#print summary of each url and store in array
summaries = []
for url in urls:
  # print(summarize_url(url['href']))
  summary = summarize_url(url['href'])
  if summary:
    summaries.append(summary)

for index, summary in enumerate(summaries):
  # print(f"{index}: {urls[index]['title']}")
  # print(f"Summary {index}: {summary}")
  # print title of given index and correspoind summary
   print(f"Summary for Title {urls[index]['title']}: (Link:{urls[index]['href']})")
   print(summary)
   print("-" * 20)



Error fetching URL https://cybernews.com/news/ai-safety-research-seriously-lacking/: 403 Client Error: Forbidden for url: https://cybernews.com/news/ai-safety-research-seriously-lacking/
Summary for Title U.S. AI Safety Institute Signs Agreements Regarding AI Safety Research ...: (Link:https://www.nist.gov/news-events/news/2024/08/us-ai-safety-institute-signs-agreements-regarding-ai-safety-research)
The U.S. AI Safety Institute has signed agreements with Anthropic and OpenAI to collaborate on AI safety research, testing, and evaluation. These agreements will provide NIST with access to major AI models prior to their public release, enabling the evaluation of capabilities, safety risks, and mitigation methods. The Institute will provide feedback to the companies on potential safety improvements, working closely with the UK AI Safety Institute. These agreements aim to advance the safe, secure, and trustworthy development and use of AI, as outlined in the Biden-Harris administration's Exe

In [ ]:
# import streamlit as st
# # Page title
# st.set_page_config(page_title='Topic Analylizer')
# st.title('Topic Analylizer')

# #query input box
# input_topic = st.text_input('Enter your Topic:', placeholder="Please enter a topic")
# input_max_result_count = st.number_input('Enter Result Number',placeholder="Enter number 10 or less")